 <img src="data/opi.jpg">

# OPI B

# B1. QQP


Descarga la Base de datos histórica de Quién es Quién en los Precios de Profeco y resuelve los
siguientes incisos. Para el procesamiento de los datos y el análisis exploratorio debes debes
usar Spark SQL, preferentemente, con Python.

## 1.- Procesamiento de los datos

## Dependencias

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import countDistinct, avg, stddev
import json
import folium
import os
import pandas as pd


In [2]:
spark = SparkSession \
    .builder \
    .appName("QQP") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
Schema = StructType([
    StructField("producto", StringType(), True),
    StructField("presentacion", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("categoria", StringType(), True), 
    StructField("catalogo", StringType(), True),
    StructField("precio", FloatType(), True),
    StructField("fechaRegistro", DateType(), True),
    StructField("cadenaComercial", StringType(), True),
    StructField("giro", StringType(), True),
    StructField("nombreComercial", StringType(), True),
    StructField("direccion", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("latitud", FloatType(), True),
    StructField("longitud", FloatType(), True),
])    
Dataframe = spark.read.csv(
    "data/all_data.csv", 
    header=True, 
schema=Schema, 
)
#show 3 rows of our DataFrame
Dataframe.show(3)

+--------------------+--------------------+--------+----------------+----------------+------+-------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+---------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud| longitud|
+--------------------+--------------------+--------+----------------+----------------+------+-------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+---------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|   2011-05-18|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.12542|
|            CRAYONES|CAJA 12 CERAS. JU...| CRAYOLA|

####   a) Cuantos registros hay?
Numero de registros:  62530715

In [4]:
num_rows = Dataframe.count()
print("Numero de registros: ", num_rows)
Dataframe.printSchema()
Dataframe.describe('precio').show()

Numero de registros:  62530715
root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: float (nullable = true)
 |-- fechaRegistro: date (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: float (nullable = true)
 |-- longitud: float (nullable = true)

+-------+------------------+
|summary|            precio|
+-------+------------------+
|  count|          62530695|
|   mean| 516.5699226315085|
| stddev|1998.6208098236448|
|    min|               0.1|
|    max|          299999.0|
+-------+------------------+



#### b) Cuantas categorias?
Hay 42 categorias

In [5]:
Dataframe.select("categoria").distinct().show()
Dataframe.select("categoria").distinct().count()

+--------------------+
|           categoria|
+--------------------+
|    MATERIAL ESCOLAR|
|ARTS. PARA EL CUI...|
|PESCADOS Y MARISC...|
|UTENSILIOS DOMEST...|
|           categoria|
|DETERGENTES Y PRO...|
|CARNE Y VISCERAS ...|
|PRODUCTOS DE TEMP...|
|GALLETAS PASTAS Y...|
|  HORTALIZAS FRESCAS|
|                null|
|  DERIVADOS DE LECHE|
|TORTILLAS Y DERIV...|
|GRASAS ANIMALES C...|
|APARATOS ELECTRON...|
|     LEGUMBRES SECAS|
|                CAFE|
|   MUEBLES DE COCINA|
|CARNES FRIAS SECA...|
|CHOCOLATES Y GOLO...|
+--------------------+
only showing top 20 rows



42

#### c) Cuantas cadenas comerciales están siendo monitoreadas?
Se están monitoreando 706 cadenas comerciales

In [6]:
Dataframe.select("cadenaComercial").distinct().show()

+--------------------+
|     cadenaComercial|
+--------------------+
|JULIO CEPEDA JUGU...|
|           WOOLWORTH|
| MARISCOS LA SEPTIMA|
|PAPELERIA EL TINTERO|
|LIBRERIA DE SANCH...|
|PAPELERIA Y LIBRE...|
|     PLAZA DEL LIBRO|
|MERCADO ALONSO FE...|
|     MERCADO LA CRUZ|
|     ZAPATERIA PAKAR|
|              SIGUBA|
|           MEGA TOYS|
| MARISCOS VILLA AZUL|
|    BODEGAS VICTORIA|
|     MINISUPER INALA|
|              S MART|
|    LIBRERIA ATHENAS|
|        FARMACIAS M+|
| MUEBLERIA CREDILAND|
|   LOYEP (UNIFORMES)|
+--------------------+
only showing top 20 rows



In [7]:
Dataframe.select("cadenaComercial").distinct().count()

706

#### d) ¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?
Haciendo un analisis explotarotio se detectaron algunas inconsistencias en los datos, en la siguiente tabla se muestran la cantidad de datos nulos.

In [8]:

Dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in Dataframe.columns]).show()


+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|producto|presentacion|marca|categoria|catalogo|precio|fechaRegistro|cadenaComercial|giro|nombreComercial|direccion|estado|municipio|latitud|longitud|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+
|       0|           0|    0|   887338|     228|    20|           20|           1184| 287|              0|        0| 15054|    15054|1167239| 1167107|
+--------+------------+-----+---------+--------+------+-------------+---------------+----+---------------+---------+------+---------+-------+--------+



####  e) ¿Cuáles son los productos más monitoreados en cada entidad?
A continuación se muestran los productos más monitoreados en cada entidad.

In [9]:
# estados = Dataframe.select("estado").distinct().toPandas().estado.tolist()
# for i in range(0, len(estados)):
#     print(estados[i])
#     df_producto_chiapas=Dataframe.filter(Dataframe.estado == estados[i]).select("producto").groupBy("producto").count().sort('count', ascending=False).show(1)

In [10]:
estados = Dataframe.select("estado").distinct().toPandas().estado.tolist()
df_productos=Dataframe.groupBy("estado","producto").count().sort('estado', ascending=False)
for i in range(0, len(estados)):
    print(estados[i])
    df_productos.filter(df_productos.estado == estados[i]).select("producto").sort('count', ascending=False).show(1)

QUINTANA ROO
+--------+
|producto|
+--------+
|     FUD|
+--------+
only showing top 1 row

NUEVO LEÓN
+-----------------+
|         producto|
+-----------------+
|DETERGENTE P/ROPA|
+-----------------+
only showing top 1 row

SINALOA
+--------+
|producto|
+--------+
|REFRESCO|
+--------+
only showing top 1 row

TABASCO
+--------+
|producto|
+--------+
|REFRESCO|
+--------+
only showing top 1 row

BAJA CALIFORNIA
+--------+
|producto|
+--------+
|REFRESCO|
+--------+
only showing top 1 row

TLAXCALA
+--------+
|producto|
+--------+
|REFRESCO|
+--------+
only showing top 1 row

COAHUILA DE ZARAGOZA
+--------+
|producto|
+--------+
|     FUD|
+--------+
only showing top 1 row

None
+--------+
|producto|
+--------+
+--------+

 ESQ. SUR 125"
+------------------+
|          producto|
+------------------+
|PAN BLANCO BOLILLO|
+------------------+
only showing top 1 row

CHIAPAS
+--------+
|producto|
+--------+
|REFRESCO|
+--------+
only showing top 1 row

 COL. EDUARDO GUERRA
+--------+
|pr

#### f)  ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?
Soriana es la cadena comercial con mayor variedad de productos monitoreados

In [11]:
df_cadena = Dataframe.groupBy("cadenaComercial","producto").count().sort('count', ascending=False)
df_cadena_max = df_cadena.groupby('cadenaComercial').agg(count('producto')).sort('count(producto)', ascending=False)
df_cadena_max.show()

+--------------------+---------------+
|     cadenaComercial|count(producto)|
+--------------------+---------------+
|             SORIANA|           1059|
|            WAL-MART|           1051|
|MEGA COMERCIAL ME...|           1049|
|  COMERCIAL MEXICANA|           1036|
|            CHEDRAUI|           1026|
|     MERCADO SORIANA|           1024|
|      BODEGA AURRERA|           1012|
|HIPERMERCADO SORIANA|           1006|
|              H.E.B.|           1001|
|        SORIANA PLUS|            999|
|       SORIANA SUPER|            996|
|BODEGA COMERCIAL ...|            979|
|        I.S.S.S.T.E.|            937|
|            SUPERAMA|            936|
|              S MART|            851|
|SUPERMERCADOS SAN...|            849|
|              SUMESA|            848|
|         CITY MARKET|            844|
|FARMACIA GUADALAJARA|            819|
|            CASA LEY|            808|
+--------------------+---------------+
only showing top 20 rows



## 2.- Análisis exploratorio

#### a) Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento

In [12]:
canasta = Dataframe.filter(Dataframe.cadenaComercial == "SUPERAMA").select("producto").distinct().toPandas().producto.tolist()
canasta_basica = ['FRIJOLES','FILETE DE RES','CAFE TOSTADO Y MOLIDO', 'COLIFLOR', 'LIMON','LECHE ULTRAPASTEURIZADA', 'PAPAYA','PAN TOSTADO',
 'MANDARINA','PATA DE CERDO']

Elegí una cadena comercial (SUPERAMA), obtuve la lista de productos y elegí 10 los cuales me parecen una canasta básica razonable, los cuales usaré para poder comprarar en esa misma cadena comercial, los precios en diferentes estados y diferentes fechas.

#### b) ¿Cuál es la ciudad más cara del país?¿Cuál es la más barata?

Usando la canasta básica de sólo 10 productos podemos calcular que en promedio, el estado más caro es VERACRUZ DE IGNACIO DE LA LLAVE
y el más barato es MICHOACÁN DE OCAMPO, pues aparecieron en el máximo y mínimo más veces.

In [19]:
for i in range(0,10):
    print(canasta_basica[i]+":")
    Dataframe.filter((Dataframe.cadenaComercial == "SUPERAMA")                         
                            &(Dataframe.producto == canasta_basica[i]) ).select("estado", "precio" ).groupBy('estado').agg(avg("precio")).sort("avg(precio)", ascending=False).show(truncate=False)


FRIJOLES:
+-------------------------------+------------------+
|estado                         |avg(precio)       |
+-------------------------------+------------------+
|VERACRUZ DE IGNACIO DE LA LLAVE|12.476923135610727|
|QUERÉTARO                      |11.548908279452261|
|MORELOS                        |11.481818216497247|
|YUCATÁN                        |11.467763152038842|
|SAN LUIS POTOSÍ                |11.441911767510806|
|DISTRITO FEDERAL               |11.355902378606647|
|GUERRERO                       |11.353333346048991|
|AGUASCALIENTES                 |11.34516130724261 |
|MÉXICO                         |11.342469733407873|
|PUEBLA                         |11.33680945123945 |
|GUANAJUATO                     |11.288230081575106|
|JALISCO                        |11.273626159835647|
|MICHOACÁN DE OCAMPO            |11.200925862347638|
+-------------------------------+------------------+

FILETE DE RES:
+-------------------+------------------+
|estado             |avg(precio)

In [20]:
maximos=['VERACRUZ DE IGNACIO DE LA LLAVE','DISTRITO FEDERAL','AGUASCALIENTES','MORELOS','VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ DE IGNACIO DE LA LLAVE',
        'GUERRERO','DISTRITO FEDERAL','QUERÉTARO','QUERÉTARO']
counts=dict()
for j in maximos:
    counts[j] = counts.get(j,0) + 1
print(counts)

{'VERACRUZ DE IGNACIO DE LA LLAVE': 3, 'DISTRITO FEDERAL': 2, 'AGUASCALIENTES': 1, 'MORELOS': 1, 'GUERRERO': 1, 'QUERÉTARO': 2}


In [21]:
minimos = ['MICHOACÁN DE OCAMPO','MICHOACÁN DE OCAMPO','VERACRUZ DE IGNACIO DE LA LLAVE',
          'MICHOACÁN DE OCAMPO','QUERÉTARO','MÉXICO','AGUASCALIENTES','AGUASCALIENTES',
          'MORELOS','VERACRUZ DE IGNACIO DE LA LLAVE']
counts = dict()
for j in minimos:
    counts[j] = counts.get(j,0) + 1
print(counts)

{'MICHOACÁN DE OCAMPO': 3, 'VERACRUZ DE IGNACIO DE LA LLAVE': 2, 'QUERÉTARO': 1, 'MÉXICO': 1, 'AGUASCALIENTES': 2, 'MORELOS': 1}


#### ACLARACION
Si hacemos el analisis por municipio la información tiene una menor correlación, es decir es dificil observar coincidencias en los precios máximos y mínimos como se muestra a continuación

In [ ]:
# for i in range(0,10):
#     print(canasta_basica[i]+":")
#     Dataframe.filter((Dataframe.cadenaComercial == "SUPERAMA")                         
#                             &(Dataframe.producto == canasta_basica[i]) 
#                            ).select("municipio", "precio" ).groupBy('municipio').agg(
#                             avg("precio")).sort("avg(precio)", ascending=False).show(truncate=False)

#### c) ¿Hay algún patrón estacional entre años?


#### d). ¿Cuál es el estado más caro y en qué mes?


#### e) ¿Cuáles son los principales riesgos de hacer análisis de series de tiempo con estos datos?
El principal riesgo es que no se registran los datos de todas las categorias de forma uniforme respecto a las fechas, por lo tanto no hay información completa, es decir, para una fecha especifica, la información no es tomada para todas las cadenas comerciales, de todos los estados, de todos los productos esto lleva a que no podamos hacer un analisis completo pues es dificil hacer coincidir las fechas en una linea temporal para todas las categorias de interés.

## 3. Visualización

#### a. Genera un mapa que nos permita identificar la oferta de categorías en la zonametropolitana de León Guanajuato y el nivel de precios en cada una de ellas. Se darán puntos extra si el mapa es interactivo

In [22]:
categorias_leon=Dataframe.filter((Dataframe.estado == "GUANAJUATO")&
                                      (Dataframe.municipio== "LEÓN")     
                                          ).select("municipio","categoria","precio"
                                                  ).groupBy("categoria").agg(avg("precio")).toPandas()
dict_leon =  dict(zip(categorias_leon.categoria.tolist(), categorias_leon["avg(precio)"].tolist()))

In [23]:
json_file = open('data/gto.json') 
gto_json = json.load(json_file)
gto_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': None,
    'state_code': 11,
    'mun_code': 9,
    'mun_name': 'Comonfort'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-100.86051359054179, 20.66102875813899],
      [-100.85195524900254, 20.681397903669357],
      [-100.8842867614841, 20.69794783441278],
      [-100.95687417676135, 20.69740223230036],
      [-100.98001339647854, 20.730502093787205],
      [-100.9670173963634, 20.77978815127586],
      [-100.98888871363035, 20.78851778507459],
      [-100.98857173801778, 20.811433073796252],
      [-100.95402139624827, 20.799247959952194],
      [-100.91376549345259, 20.807795726380114],
      [-100.8896753468977, 20.793428204086375],
      [-100.80377495589276, 20.80815946112173],
      [-100.78634129720173, 20.78215242709635],
      [-100.71819154050038, 20.831256617214198],
      [-100.67761866209216, 20.79997542943542],
      [-100.63229114949546, 20.792336999861533],
      [-100.

In [24]:
indice_leon=0
for j in range(0,len(gto_json['features'])):
    if gto_json['features'][j]['properties']['mun_name']=='Leon':
        indice_leon=j
        print(indice_leon)

41


In [25]:
gto_json['features'][j]['properties']= gto_json['features'][41]['properties'].update(dict_leon)
gto_json['features']=[gto_json['features'][indice_leon]]
with open('data/gto_2.json', 'w') as json_file:
  json.dump(gto_json, json_file)

## Elegí el municipio de León y sólo 20 productos para esta visualización

In [26]:
path='data'
overlay = os.path.join('data', 'gto_2.json')
#Base
m = folium.Map(
location = [21.145453, -101.603676], zoom_start=10)
colonias_geo = folium.GeoJson(   
     overlay,     
     name= 'Guanajuato',         ).add_to(m)
# folium.GeoJson(overlay, name= 'Guanajuato').add_to(m)
folium.features.GeoJsonTooltip(fields=["mun_name"]+categorias_leon.dropna().categoria.tolist()[0:20], 
                               aliases=['Municipio'] +categorias_leon.dropna().categoria.tolist()[0:20]).add_to(colonias_geo)
folium.LayerControl().add_to(m)
m